In [ ]:
# Importing necessary packages and tools
import os, sys, itertools, numpy as np, tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
#Defining variables and folder paths
DATA_DIR = "DATA/combined_images"         # root containing subfolders per class
IMG_SIZE = (150, 150)
BATCH = 64                    # try 64 first; drop to 32 if memory is tight
EPOCHS = 20
VAL_SPLIT = 0.2               # train/val split from the directory
SEED = 42

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VAL_SPLIT,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH,
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VAL_SPLIT,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH,
)

class_names = train_ds.class_names
num_classes = len(class_names)
print("Classes:", class_names)


In [ ]:
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),     # for ASL: keep if your dataset is not direction-sensitive; otherwise remove
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.05),
    layers.RandomBrightness(factor=0.1),
])

In [ ]:
def make_model(input_shape=(150,150,3), num_classes=18, dropout=0.4):
    inputs = keras.Input(shape=input_shape)

    x = data_augmentation(inputs)
    x = layers.Rescaling(1./255)(x)

    def conv_block(x, filters):
        x = layers.Conv2D(filters, 3, padding="same", use_bias=False)(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
        x = layers.Conv2D(filters, 3, padding="same", use_bias=False)(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
        return x

    x = conv_block(x, 32)
    x = layers.MaxPooling2D()(x)

    x = conv_block(x, 64)
    x = layers.MaxPooling2D()(x)

    x = conv_block(x, 128)
    x = layers.MaxPooling2D()(x)

    x = conv_block(x, 256)
    x = layers.MaxPooling2D()(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs, name="asl_cnn")

model = make_model(input_shape=IMG_SIZE + (3,), num_classes=num_classes)

# Optimizer & compile
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("asl_cnn_best.keras", save_best_only=True, monitor="val_accuracy"),
    keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, monitor="val_accuracy"),
    keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, min_lr=1e-5, monitor="val_loss")
]

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)

In [ ]:
eval_res = model.evaluate(val_ds, verbose=0)
print(f"Validation — loss: {eval_res[0]:.4f}  acc: {eval_res[1]:.4f}")

In [ ]:
try:
    from sklearn.metrics import confusion_matrix, classification_report
    y_true, y_pred = [], []
    for images, labels in val_ds:
        preds = model.predict(images, verbose=0)
        y_true.extend(labels.numpy().tolist())
        y_pred.extend(np.argmax(preds, axis=1).tolist())

    cm = confusion_matrix(y_true, y_pred)
    print("\nClassification report:\n",
          classification_report(y_true, y_pred, target_names=class_names))
    # simple text cm
    print("\nConfusion matrix (rows=true, cols=pred):\n", cm)
except Exception as e:
    print("Install scikit-learn for confusion matrix: pip install scikit-learn")
    print("Skipped confusion matrix.", e)

In [ ]:
model.save("asl_cnn_final.keras")
print("Saved model to asl_cnn_final.keras")